In [1]:
import numpy as onp
import tensorflow as tf
import matplotlib.pyplot as plt

from utils import *

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

2 Physical GPUs, 1 Logical GPU


# hyperparameters

In [3]:
#data
DATASET = 'cifar10'
class_num   = 10
test_size   = 2048
train_size  = None
image_shape = None

if DATASET =='mnist':
    image_shape = (28, 28, 1)
elif DATASET == 'cifar10':
    image_shape = (32, 32, 3)

#training
run = 10
batch_size = 256
epochs = 1000

In [4]:
x_train_all, y_train_all, x_test_all, y_test_all = tuple(onp.array(x) for x in get_dataset(DATASET, None, None, 
                                                                                  do_flatten_and_normalize=False))

In [5]:
# shuffle
seed = 0
x_train_all, y_train_all = shaffle(x_train_all, y_train_all, seed)

In [6]:
# down sample
x_train = x_train_all[:train_size]
y_train = y_train_all[:train_size]

x_test = x_test_all[:test_size]
y_test = y_test_all[:test_size]

In [7]:
x_train, x_test = x_train.reshape((-1, *image_shape)), x_test.reshape((-1, *image_shape))

In [8]:
layers = tf.keras.layers

In [9]:
gaussian_initializer=tf.keras.initializers.GlorotNormal()

In [10]:
img_input = layers.Input(shape=image_shape)
x = layers.Conv2D(64,  (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(img_input)
x = layers.Conv2D(64,  (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=gaussian_initializer)(x)
x = layers.MaxPool2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation='relu', kernel_initializer=gaussian_initializer)(x)
x = layers.Dense(256, activation='relu', kernel_initializer=gaussian_initializer)(x)
x = layers.Dense(256, activation='relu', kernel_initializer=gaussian_initializer)(x)
out = layers.Dense(10)(x)

model = tf.keras.Model(inputs=img_input, outputs=out)

In [11]:
def scheduler(epoch, lr):
    if epoch < 5:
        return 1e-4
    elif epoch < 55:
        return 1e-2
    elif epoch < 105:
        return 1e-2
    else:
        return 1e-4

In [12]:
model.compile(optimizer=tf.keras.optimizers.SGD(0.01),
              loss=tf.keras.losses.MSE,
              metrics=['accuracy'])

In [13]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[callback])

Epoch 1/1000
176/176 [==============================] - 13s 75ms/step - loss: 0.0999 - accuracy: 0.1016 - val_loss: 0.0998 - val_accuracy: 0.0938
Epoch 2/1000
176/176 [==============================] - 6s 35ms/step - loss: 0.0998 - accuracy: 0.1007 - val_loss: 0.0997 - val_accuracy: 0.0938
Epoch 3/1000
176/176 [==============================] - 8s 47ms/step - loss: 0.0996 - accuracy: 0.1007 - val_loss: 0.0996 - val_accuracy: 0.0938
Epoch 4/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0995 - accuracy: 0.1007 - val_loss: 0.0994 - val_accuracy: 0.0938
Epoch 5/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0993 - accuracy: 0.1007 - val_loss: 0.0993 - val_accuracy: 0.0938
Epoch 6/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0945 - accuracy: 0.1007 - val_loss: 0.0917 - val_accuracy: 0.0938
Epoch 7/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0908 - accuracy: 0.1007 - val_loss: 0.0903 - val_a

176/176 [==============================] - 6s 32ms/step - loss: 0.0900 - accuracy: 0.1140 - val_loss: 0.0900 - val_accuracy: 0.0948
Epoch 58/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0900 - accuracy: 0.1196 - val_loss: 0.0900 - val_accuracy: 0.1096
Epoch 59/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0900 - accuracy: 0.1174 - val_loss: 0.0900 - val_accuracy: 0.0946
Epoch 60/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0900 - accuracy: 0.1098 - val_loss: 0.0900 - val_accuracy: 0.0938
Epoch 61/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0900 - accuracy: 0.1163 - val_loss: 0.0900 - val_accuracy: 0.0942
Epoch 62/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0900 - accuracy: 0.1130 - val_loss: 0.0900 - val_accuracy: 0.0930
Epoch 63/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0900 - accuracy: 0.1167 - val_loss: 0.0900 - val_accuracy:

176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1521 - val_loss: 0.0899 - val_accuracy: 0.1494
Epoch 114/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1521 - val_loss: 0.0899 - val_accuracy: 0.1498
Epoch 115/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1524 - val_loss: 0.0899 - val_accuracy: 0.1496
Epoch 116/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1522 - val_loss: 0.0899 - val_accuracy: 0.1494
Epoch 117/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1525 - val_loss: 0.0899 - val_accuracy: 0.1496
Epoch 118/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1519 - val_loss: 0.0899 - val_accuracy: 0.1502
Epoch 119/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1521 - val_loss: 0.0899 - val_acc

Epoch 169/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1513 - val_loss: 0.0899 - val_accuracy: 0.1482
Epoch 170/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1505 - val_loss: 0.0899 - val_accuracy: 0.1480
Epoch 171/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1520 - val_loss: 0.0899 - val_accuracy: 0.1476
Epoch 172/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1517 - val_loss: 0.0899 - val_accuracy: 0.1474
Epoch 173/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1516 - val_loss: 0.0899 - val_accuracy: 0.1474
Epoch 174/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1520 - val_loss: 0.0899 - val_accuracy: 0.1474
Epoch 175/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1516 - val_loss: 0

Epoch 225/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1538 - val_loss: 0.0899 - val_accuracy: 0.1486
Epoch 226/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1537 - val_loss: 0.0899 - val_accuracy: 0.1484
Epoch 227/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1545 - val_loss: 0.0899 - val_accuracy: 0.1486
Epoch 228/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1537 - val_loss: 0.0899 - val_accuracy: 0.1486
Epoch 229/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1542 - val_loss: 0.0899 - val_accuracy: 0.1490
Epoch 230/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1550 - val_loss: 0.0899 - val_accuracy: 0.1490
Epoch 231/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1543 - val_loss: 0

...

Epoch 617/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1628 - val_loss: 0.0899 - val_accuracy: 0.1556
Epoch 618/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1619 - val_loss: 0.0899 - val_accuracy: 0.1558
Epoch 619/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1620 - val_loss: 0.0899 - val_accuracy: 0.1560
Epoch 620/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1619 - val_loss: 0.0899 - val_accuracy: 0.1558
Epoch 621/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1617 - val_loss: 0.0899 - val_accuracy: 0.1560
Epoch 622/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1622 - val_loss: 0.0899 - val_accuracy: 0.1560
Epoch 623/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1618 - val_loss: 0

Epoch 673/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1619 - val_loss: 0.0899 - val_accuracy: 0.1544
Epoch 674/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1608 - val_loss: 0.0899 - val_accuracy: 0.1544
Epoch 675/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1614 - val_loss: 0.0899 - val_accuracy: 0.1544
Epoch 676/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1613 - val_loss: 0.0899 - val_accuracy: 0.1546
Epoch 677/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1617 - val_loss: 0.0899 - val_accuracy: 0.1546
Epoch 678/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1619 - val_loss: 0.0899 - val_accuracy: 0.1546
Epoch 679/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1608 - val_loss: 0

Epoch 729/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1629 - val_loss: 0.0899 - val_accuracy: 0.1550
Epoch 730/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1620 - val_loss: 0.0899 - val_accuracy: 0.1550
Epoch 731/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1613 - val_loss: 0.0899 - val_accuracy: 0.1550
Epoch 732/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1610 - val_loss: 0.0899 - val_accuracy: 0.1550
Epoch 733/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1614 - val_loss: 0.0899 - val_accuracy: 0.1550
Epoch 734/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1617 - val_loss: 0.0899 - val_accuracy: 0.1552
Epoch 735/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1624 - val_loss: 0

Epoch 785/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1623 - val_loss: 0.0899 - val_accuracy: 0.1554
Epoch 786/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1630 - val_loss: 0.0899 - val_accuracy: 0.1554
Epoch 787/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1631 - val_loss: 0.0899 - val_accuracy: 0.1552
Epoch 788/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1627 - val_loss: 0.0899 - val_accuracy: 0.1554
Epoch 789/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1633 - val_loss: 0.0899 - val_accuracy: 0.1552
Epoch 790/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1634 - val_loss: 0.0899 - val_accuracy: 0.1554
Epoch 791/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1627 - val_loss: 0

Epoch 841/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1636 - val_loss: 0.0899 - val_accuracy: 0.1562
Epoch 842/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1631 - val_loss: 0.0899 - val_accuracy: 0.1560
Epoch 843/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1626 - val_loss: 0.0899 - val_accuracy: 0.1560
Epoch 844/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1640 - val_loss: 0.0899 - val_accuracy: 0.1560
Epoch 845/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1634 - val_loss: 0.0899 - val_accuracy: 0.1562
Epoch 846/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1632 - val_loss: 0.0899 - val_accuracy: 0.1562
Epoch 847/1000
176/176 [==============================] - 6s 32ms/step - loss: 0.0899 - accuracy: 0.1632 - val_loss: 0

In [ ]:
model.save('./model_weights/vgg_19_train=None_mse_init=gaussian.h5')